In [1]:
# assignment 6
# MSDS 422
# Professor Anil Chaturvedi

# https://github.com/ageron/handson-ml/blob/master/10_introduction_to_artificial_neural_networks.ipynb
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
# https://github.com/ageron/handson-ml/blob/master/12_distributed_tensorflow.ipynb

# more info
# https://www.datacamp.com/community/tutorials/cnn-tensorflow-python
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

# really wonderful resource for noobs
# https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

<div style="text-align: right"><b>pkg imports</b></div>

In [2]:
# basix
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import time

# prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#modeling
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense

# other

%matplotlib inline   
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<div style="text-align: right"><b>configs, table setup</b></div>

In [3]:
"""
for sci-kit:
‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

for tf/keras, the same, respectively:
linear
sigmoid
tanh
relu

also (with tf):
selu
"""

#sk_act_fun = "identity"
#sk_act_fun = "logistic"
sk_act_fun = "tanh"
#sk_act_fun = "relu"

tf_act_fun = "tanh"

In [4]:
pd.options.display.float_format = "{:,.4f}".format
RANDOM_SEED = 8675309
responseVar = "label"

In [5]:
# setup for consolidated table, later

table = {"model 1":{"pkg":"sci-kit","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 2":{"pkg":"sci-kit","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 3":{"pkg":"sci-kit","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 4":{"pkg":"sci-kit","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 5":{"pkg":"tf/keras","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 6":{"pkg":"tf/keras","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 7":{"pkg":"tf/keras","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 8":{"pkg":"tf/keras","act'n fn":"tanh","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None}
        }

pd.DataFrame(table).transpose().head(2)

,pkg,act'n fn,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,sci-kit,tanh,None,None,None,None,None
model 2,sci-kit,tanh,None,None,None,None,None


<div style="text-align: right"><b>defining things for model</b></div>

In [6]:
# activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# lambda x: 1 / (1 + np.exp(-x))

In [7]:
training_iters = 10
learning_rate = 0.01
b_size = 128

# MNIST data input (img shape: 28*28)
n_input = 28

# MNIST total classes (0-9 digits)
n_classes = 10

<div style="text-align: right"><b>data setup for scikit</b></div>

In [8]:
sk_mnist_train = pd.read_csv('train.csv')
sk_mnist_test = pd.read_csv('test.csv')
#sk_mnist = pd.concat([mnist_test, mnist_train.drop(columns = "label")]) # combines the rows of 2 datasets

In [9]:
print("le shape of le train is le : ", sk_mnist_train.shape)
print("le shape of le test        : ", sk_mnist_test.shape)

le shape of le train is le :  (42000, 785)
le shape of le test        :  (28000, 784)


In [10]:
# 'test' data is our future data
sk_future_data = sk_mnist_test
sk_mnist = sk_mnist_train
del(sk_mnist_test)
del(sk_mnist_train)

In [11]:
try:
    sk_mnist_test
except NameError as e:
    print(e) # meow
    
try:
    sk_mnist_train
except NameError as e:
    print(e) # meow

name 'sk_mnist_test' is not defined
name 'sk_mnist_train' is not defined


In [12]:
#X = mnist_scaled.loc[:, mnist_scaled.columns != responseVar]
#y = mnist_scaled[responseVar]
sk_X = sk_mnist.loc[:, sk_mnist.columns != responseVar]
sk_y = sk_mnist[responseVar]

In [13]:
pixel744 = pd.DataFrame(sk_mnist.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

In [14]:
scaler = StandardScaler()
sk_np_mnist_x = scaler.fit_transform(sk_X)
sk_X_scaled = pd.DataFrame(sk_np_mnist_x, index=sk_X.index, columns=sk_X.columns)

In [15]:
pd.DataFrame(sk_X_scaled.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

,pixel774
count,"42,000.0000"
mean,0.0000
std,1.0000
min,-0.0347
max,40.2009


In [16]:
sk_X_train, sk_X_test, sk_y_train, sk_y_test = train_test_split(sk_X_scaled, sk_y,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=RANDOM_SEED)

<div style="text-align: right"><b>data setup for tensorflow</b></div>

In [17]:
from keras.datasets import mnist
(tf_X_train, tf_y_train), (tf_X_test, tf_y_test) = mnist.load_data()

In [18]:
tf_X_valid, tf_X_train = tf_X_train[:5000] / 255.0, tf_X_train[5000:] / 255.0
tf_y_valid, tf_y_train = tf_y_train[:5000], tf_y_train[5000:]
tf_X_test = tf_X_test / 255.0

<div style="text-align: right"><b>model 1 : 2x1. 2 layers w 1 neuron each. sci-kit</b></div>

In [19]:
print("approximate start time : ", time.asctime())
start1 = time.perf_counter()

approximate start time :  Sat Oct 24 14:25:37 2020


In [20]:
twox1 = MLPClassifier(hidden_layer_sizes = (2, 1), # neurons comma layers
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [21]:
stop1 = time.perf_counter()
tm1 = round(stop1 - start1)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm1)
print("time of execution in minutes : ", round(tm1/60, 1))

approximate completion time :  Sat Oct 24 14:26:30 2020
time of execution in seconds :  53
time of execution in minutes :  0.9


In [22]:
table["model 1"]["nbr_layers"] = "1"
table["model 1"]["nodes_p_layer"] = "2"
table["model 1"]["processing_time"] = tm1
table["model 1"]["testset_acc"] = twox1.score(sk_X_test, sk_y_test)
table["model 1"]["trainset_acc"] = twox1.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 2 : 2x2. 2 layers w 2 neurons each. I think. sci-kit</b></div>

In [23]:
print("approximate start time : ", time.asctime())
start2 = time.perf_counter()

approximate start time :  Sat Oct 24 14:26:30 2020


In [24]:
twox2 = MLPClassifier(hidden_layer_sizes = (2, 2), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [25]:
stop2 = time.perf_counter()
tm2 = round(stop2 - start2)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm2)
print("time of execution in minutes : ", round(tm2/60, 1))

approximate completion time :  Sat Oct 24 14:27:21 2020
time of execution in seconds :  51
time of execution in minutes :  0.8


In [26]:
table["model 2"]["nbr_layers"] = "2"
table["model 2"]["nodes_p_layer"] = "2"
table["model 2"]["processing_time"] = tm2
table["model 2"]["testset_acc"] = twox2.score(sk_X_test, sk_y_test)
table["model 2"]["trainset_acc"] = twox2.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 3 : x1. 30 layers w 4 neurons each. sci-kit</b></div>

In [27]:
print("approximate start time : ", time.asctime())
start3 = time.perf_counter()

approximate start time :  Sat Oct 24 14:27:21 2020


In [28]:
thirtyx4 = MLPClassifier(hidden_layer_sizes = (30, 4), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

In [29]:
stop3 = time.perf_counter()
tm3 = round(stop3 - start3)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm3)
print("time of execution in minutes : ", round(tm3/60, 1))

approximate completion time :  Sat Oct 24 14:28:12 2020
time of execution in seconds :  51
time of execution in minutes :  0.8


In [30]:
table["model 3"]["nbr_layers"] = "30"
table["model 3"]["nodes_p_layer"] = "4"
table["model 3"]["processing_time"] = tm3
table["model 3"]["testset_acc"] = thirtyx4.score(sk_X_test, sk_y_test)
table["model 3"]["trainset_acc"] = thirtyx4.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 4 : x1. 4 layers, each w 30 neurons. sci-kit</b></div>

In [31]:
print("approximate start time : ", time.asctime())
start4 = time.perf_counter()

approximate start time :  Sat Oct 24 14:28:12 2020


In [32]:
fourx30 = MLPClassifier(hidden_layer_sizes = (4, 30), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

In [33]:
stop4 = time.perf_counter()
tm4 = round(stop4 - start4)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm4)
print("time of execution in minutes : ", round(tm4/60, 1))

approximate completion time :  Sat Oct 24 14:29:00 2020
time of execution in seconds :  47
time of execution in minutes :  0.8


In [34]:
table["model 4"]["nbr_layers"] = "4"
table["model 4"]["nodes_p_layer"] = "30"
table["model 4"]["processing_time"] = tm4
table["model 4"]["testset_acc"] = fourx30.score(sk_X_test, sk_y_test)
table["model 4"]["trainset_acc"] = fourx30.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 5 : 2x1. 2 layers w 1 neurons each</b></div>

In [35]:
print("approximate start time : ", time.asctime())
start5 = time.perf_counter()

approximate start time :  Sat Oct 24 14:29:00 2020


In [36]:
#classification MLP with two hidden layers
model5 = Sequential()
model5.add(Flatten(input_shape=[28, 28]))
model5.add(Dense(1, activation=tf_act_fun))
model5.add(Dense(1, activation=tf_act_fun))
model5.add(Dense(10, activation="softmax"))

In [37]:
model5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 785       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20        
Total params: 807
Trainable params: 807
Non-trainable params: 0
_________________________________________________________________


In [38]:
model5.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [39]:
history5 = model5.fit(tf_X_train, tf_y_train, epochs=5,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.1349 - accuracy: 0.1876 - val_loss: 2.0322 - val_accuracy: 0.2144
Epoch 2/5
1719/1719 [==============================] - 2s 988us/step - loss: 1.9635 - accuracy: 0.2116 - val_loss: 1.9106 - val_accuracy: 0.2056
Epoch 3/5
1719/1719 [==============================] - 2s 990us/step - loss: 1.8866 - accuracy: 0.1969 - val_loss: 1.8642 - val_accuracy: 0.1850
Epoch 4/5
1719/1719 [==============================] - 2s 984us/step - loss: 1.8556 - accuracy: 0.2106 - val_loss: 1.8439 - val_accuracy: 0.2128
Epoch 5/5
1719/1719 [==============================] - 2s 986us/step - loss: 1.8402 - accuracy: 0.2189 - val_loss: 1.8315 - val_accuracy: 0.2198


In [40]:
stop5 = time.perf_counter()
tm5 = round(stop5 - start5)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm5)
print("time of execution in minutes : ", round(tm5/60, 1))

approximate completion time :  Sat Oct 24 14:29:09 2020
time of execution in seconds :  9
time of execution in minutes :  0.1


In [41]:
table["model 5"]["nbr_layers"] = "2"
table["model 5"]["nodes_p_layer"] = "1"
table["model 5"]["processing_time"] = tm5
table["model 5"]["testset_acc"] = model5.evaluate(tf_X_test, tf_y_test)[1]
table["model 5"]["trainset_acc"] = model5.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 1s 818us/step - loss: 1.8308 - accuracy: 0.2240


<div style="text-align: right"><b>model 6 : 2x2. 2 layers w 2 neurons each</b></div>

In [42]:
print("approximate start time : ", time.asctime())
start6 = time.perf_counter()

approximate start time :  Sat Oct 24 14:29:11 2020


In [43]:
#classification MLP with two hidden layers
model6 = Sequential()
model6.add(Flatten(input_shape=[28, 28]))
model6.add(Dense(2, activation=tf_act_fun))
model6.add(Dense(2, activation=tf_act_fun))
model6.add(Dense(10, activation="softmax"))

In [44]:
model6.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1570      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                30        
Total params: 1,606
Trainable params: 1,606
Non-trainable params: 0
_________________________________________________________________


In [45]:
model6.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history6 = model6.fit(tf_X_train, tf_y_train, epochs=5,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.0115 - accuracy: 0.2944 - val_loss: 1.7844 - val_accuracy: 0.4446
Epoch 2/5
1719/1719 [==============================] - 2s 1ms/step - loss: 1.6570 - accuracy: 0.4724 - val_loss: 1.5179 - val_accuracy: 0.5068
Epoch 3/5
1719/1719 [==============================] - 2s 1ms/step - loss: 1.4667 - accuracy: 0.4930 - val_loss: 1.3901 - val_accuracy: 0.5158
Epoch 4/5
1719/1719 [==============================] - 2s 1ms/step - loss: 1.3762 - accuracy: 0.5002 - val_loss: 1.3348 - val_accuracy: 0.5286
Epoch 5/5
1719/1719 [==============================] - 2s 1ms/step - loss: 1.3286 - accuracy: 0.5167 - val_loss: 1.2933 - val_accuracy: 0.5304


In [46]:
stop6 = time.perf_counter()
tm6 = round(stop6 - start6)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm6)
print("time of execution in minutes : ", round(tm6/60, 1))

approximate completion time :  Sat Oct 24 14:29:20 2020
time of execution in seconds :  9
time of execution in minutes :  0.1


In [47]:
table["model 6"]["nbr_layers"] = "2"
table["model 6"]["nodes_p_layer"] = "2"
table["model 6"]["processing_time"] = tm6
table["model 6"]["testset_acc"] = model6.evaluate(tf_X_test, tf_y_test)[1]
table["model 6"]["trainset_acc"] = model6.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 1s 809us/step - loss: 1.3054 - accuracy: 0.5232


<div style="text-align: right"><b>model 7 : 30x4. 30 layers w 4 neurons each</b></div>

In [48]:
print("approximate start time : ", time.asctime())
start7 = time.perf_counter()

approximate start time :  Sat Oct 24 14:29:22 2020


In [49]:
#classification MLP with two hidden layers
model7 = Sequential()
model7.add(Flatten(input_shape=[28, 28]))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(10, activation="softmax"))

In [50]:
model7.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 3140      
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_11 (Dense)             (None, 4)                

In [51]:
model7.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history7 = model7.fit(tf_X_train, tf_y_train, epochs=30,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.2528 - accuracy: 0.1275 - val_loss: 2.0071 - val_accuracy: 0.1866
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 1.9423 - accuracy: 0.2024 - val_loss: 1.8940 - val_accuracy: 0.2094
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 1.8806 - accuracy: 0.2083 - val_loss: 1.8484 - val_accuracy: 0.2026
Epoch 4/30
1719/1719 [==============================] - 3s 2ms/step - loss: 1.8405 - accuracy: 0.2088 - val_loss: 1.8212 - val_accuracy: 0.2110
Epoch 5/30
1719/1719 [==============================] - 3s 2ms/step - loss: 1.8152 - accuracy: 0.2131 - val_loss: 1.7994 - val_accuracy: 0.2178
Epoch 6/30
1719/1719 [==============================] - 3s 2ms/step - loss: 1.7939 - accuracy: 0.2256 - val_loss: 1.7644 - val_accuracy: 0.2660
Epoch 7/30
1719/1719 [==============================] - 3s 2ms/step - loss: 1.7434 - accuracy: 0.2747 - val_loss: 1.6704 - val_accuracy:

In [52]:
stop7 = time.perf_counter()
tm7 = round(stop7 - start7)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm7)
print("time of execution in minutes : ", round(tm7/60, 1))

approximate completion time :  Sat Oct 24 14:30:51 2020
time of execution in seconds :  89
time of execution in minutes :  1.5


In [53]:
table["model 7"]["nbr_layers"] = "30"
table["model 7"]["nodes_p_layer"] = "4"
table["model 7"]["processing_time"] = tm7
table["model 7"]["testset_acc"] = model7.evaluate(tf_X_test, tf_y_test)[1]
table["model 7"]["trainset_acc"] = model7.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 2s 1ms/step - loss: 1.1736 - accuracy: 0.4798: 0s - loss: 1.1765 - accuracy: 0.


<div style="text-align: right"><b>model 8 : 4x30. 4 layers w 30 neurons each</b></div>

In [54]:
print("approximate start time : ", time.asctime())
start8 = time.perf_counter()

approximate start time :  Sat Oct 24 14:30:53 2020


In [55]:
#classification MLP with two hidden layers
model8 = Sequential()
model8.add(Flatten(input_shape=[28, 28]))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(10, activation="softmax"))

In [56]:
model8.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 30)                23550     
_________________________________________________________________
dense_37 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_38 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_39 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_40 (Dense)             (None, 10)                310       
Total params: 26,650
Trainable params: 26,650
Non-trainable params: 0
__________________________________________________

In [57]:
model8.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history8 = model7.fit(tf_X_train, tf_y_train, epochs=12,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/12
1719/1719 [==============================] - 3s 2ms/step - loss: 1.3357 - accuracy: 0.4394 - val_loss: 1.6664 - val_accuracy: 0.3456
Epoch 2/12
1719/1719 [==============================] - 3s 2ms/step - loss: 1.2448 - accuracy: 0.4612 - val_loss: 1.1446 - val_accuracy: 0.4898
Epoch 3/12
1719/1719 [==============================] - 3s 2ms/step - loss: 1.3701 - accuracy: 0.4372 - val_loss: 1.1680 - val_accuracy: 0.4852
Epoch 4/12
1719/1719 [==============================] - 3s 2ms/step - loss: 1.1726 - accuracy: 0.4795 - val_loss: 1.1423 - val_accuracy: 0.4870
Epoch 5/12
1719/1719 [==============================] - 3s 2ms/step - loss: 1.1588 - accuracy: 0.4815 - val_loss: 1.1254 - val_accuracy: 0.4922
Epoch 6/12
1719/1719 [==============================] - 3s 2ms/step - loss: 1.1827 - accuracy: 0.4808 - val_loss: 1.1675 - val_accuracy: 0.4754
Epoch 7/12
1719/1719 [==============================] - 3s 2ms/step - loss: 1.3739 - accuracy: 0.4441 - val_loss: 1.3959 - val_accuracy:

In [58]:
stop8 = time.perf_counter()
tm8 = round(stop8 - start8)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm8)
print("time of execution in minutes : ", round(tm8/60, 1))

approximate completion time :  Sat Oct 24 14:31:28 2020
time of execution in seconds :  35
time of execution in minutes :  0.6


In [59]:
table["model 8"]["nbr_layers"] = "4"
table["model 8"]["nodes_p_layer"] = "30"
table["model 8"]["processing_time"] = tm7
table["model 8"]["testset_acc"] = model8.evaluate(tf_X_test, tf_y_test)[1]
table["model 8"]["trainset_acc"] = model8.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 1s 860us/step - loss: 2.3854 - accuracy: 0.10060s


In [60]:
print("processing time in seconds : ")
pd.DataFrame(table).transpose()

processing time in seconds : 


,pkg,act'n fn,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,sci-kit,tanh,1,2,53,0.3876,0.3801
model 2,sci-kit,tanh,2,2,51,0.5874,0.5689
model 3,sci-kit,tanh,30,4,51,0.9919,0.8949
model 4,sci-kit,tanh,4,30,47,0.8855,0.8497
model 5,tf/keras,tanh,2,1,9,0.2240,0.2229
model 6,tf/keras,tanh,2,2,9,0.5232,0.5206
model 7,tf/keras,tanh,30,4,89,0.4798,0.4735
model 8,tf/keras,tanh,4,30,89,0.1006,0.1001
